# Retke matrice

Podsetimo se da su retke matrice (engl. sparse matrix) matrice u kojima je *veliki broj* elementa jednak nuli. 
![retke matrice](assets/sparse_matrix.png)
Zahvaljujući ovoj svojoj osobini za njihovo čuvanje je potrebno manje memorije, a operacije nad njima se mogu izvoditi efikasnije. Ova sveska sumira osnovne formate retkih matrica i neke principe rada sa njima.

Matrice koje nisu retke zovemo gustim matricama (engl. dense matrix).

In [1]:
import numpy as np
np.random.seed(7)

In [2]:
from matplotlib import pyplot as plt 

Rad sa retkim matricama podržan je paketom `sparse` biblioteke `scipy`.

In [3]:
from scipy import sparse

Osnovni formati za zapis retkih matrica su: `CSC` (Compressed Sparse Column), `CSR` (Compressed Sparse Row), `COO` (COOrdinate format), `DOK` (Dictionary of Keys), `DIA` (DIAgonal format), `LiL` (List of Lists) i `BSR` (Block Compressed Row). Nadalje ćemo upoznati neke najčešće korišćene.

U `COO` formatu matrice se čuvaju kao trojke indeksa vrsta, indeksa kolona i vrednosti elementa različitih od nule. <img src='assets/coo_format.gif' style='width:600px'>

U `CSR` formatu čuvaju se indeksi vrsta, indeksi kolona i vrednosti elemenata različitih od nule ali sa idejom da se dodatno optimizuju operacije nad njima. Za svaka dva susedna elementa u indeksu vrsta prvi broj označava indeks vrste u kojoj se nalaze ne-nula elementi, a drugi je izračunat tako da njihova razlika predstavlja broj ne-nula elemenata u vrsti. U indeksu kolona popisane su za svaku vrstu uzastopno indeksi kolona u kojima se nalaze ne-nula elementi. Niz sa vrednostima elemenata sadrži same elemente. Sledeća animacija približava ovaj (možda malo konfuzan) opis. <img src='assets/csr_format.gif' style='width:600px'>

`CSC` format predstavlja ekvivalent prethodnog formata ali sa specifično zapisanim indeksom kolona. <img src='assets/csc_format.gif' style='width:600px'>

Format `DIA` je specijalizovan za čuvanje retkih dijagonalnih matrica. <img src='assets/dia_format.gif' style='width:600px'>

Biblioteka podržava konverzije iz jednog formata zapisa u drugi format. Mogu se koristiti funkcije koje smo pomenuli, ali i funkcije `tocoo`, `tocsc`, `tocsr`, ...

Kao što smo videli u primerima, gusta reprezentacija retkih matrica se može dobiti pozivima `toarray` ili `todense` metoda. Prva metoda vraća matricu koja je tipa `ndarray`, a druga matricu koja je tipa `matrix`. Tip `matrix` se razlikuje od `ndarray` niza po specifičnostima nekih operacija, na primer A\*B predstavlja matrično, a ne pokoordinatno množenje. Preporučuje se korišćenje `toarray` metode.

Ukoliko je od postojeće guste matrice potrebno kreirati retku matricu, mogu se iskoristiti funkcije `coo_matrix`, `csr_matrix` i `csc_matrix` i njima slične.

In [28]:
array = np.array([
    [ 1,  0, 13,  0,  0],
    [ 0,  2,  0, 14,  0],
    [ 0,  0,  3,  0, 15],
    [ 6,  0,  0,  4,  0],
    [ 0,  7,  0,  0,  5],
    [ 0,  0,  8,  0,  0],
    [ 0,  0,  0,  9,  0]])

In [29]:
sparse_array = sparse.csc_matrix(array)

Proizvoljne retke matrice se mogu generisati pozivom funkcije `random` koja očekuje informacije o dimenziji matrice i gustini matice. Gustina matrice je realan broj u intervalu [0, 1] gde vrednost 0 predstavlja matricu nula, a 1 sasvim gustu matricu (bez nula vrednosti). Alternativno, može se koristiti i funkcija `rand` koja bira vrednosti iz uniformne raspodele. Podrazumevani format matrica je COO, a može se promeniti navođenjem parametra `format`.

Broj ne-nula (engl. non-zero) elemenata se može dobiti svojstvom `nnz` ili pozivom funkcije `count_nonzero`, a sami indeksi elemenata pozivom funkcije `nonzero`.

Na osnovu broja ne-nula vrednosti se može izraćunati koeficijent retkosti matrice po formuli $1 - \frac{broj\ ne-nula\ elemenata\ matrice}{ukupan\ broj\ elemenata\ matrice}$.

Za vizuelizaciju retkih matrica može se iskoristiti funkcija `spy` biblioteke `matplotlib`. Ova funkcija prikazuje samo vrednosti različite od nule na odgovarajućoj 2D mreži čije dimenzije odgovaraju dimenziji matrice pa se uz brojčane vrednosti može steći utisak o strukturi matrice. 

Uporedimo razlike u veličinama gustih i retkih matrica. Skup podataka sa kojim ćemo eksperimentisati će biti uzorkovan iz binomne raspodele sa verovatnoćom uspeha 0.1 i biće dimenzija 2000x10000.  

Uporedimo sada i vremensku dimenziju rada sa gustim i retkim matricama. Matrice su istih oblika (imaju istu vrednost `shape` svojstva, ali imaju različite veličine tj. različit broj elemenata koji možemo da pročitamo svojstvom `size`).

Koristićemo paket `time` za merenje vremena potrebnog za izvršavanje algebarskih operacija. Pratićemo izračunavanje proizvoda $MM^T$ zadate matrice $M$.

Na nivou `linalg` paketa biblioteke `scipy` su podržane algebarske funkcije za rad nad ovakvim tipom matrica. 

In [4]:
from scipy.sparse import linalg